In [1]:
import mysql.connector
import pandas as pd

from IPython.display import display, Markdown, HTML

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo",
    charset="latin1"
)

/home/kulla/.local/share/asdf/installs/python/3.9.1/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [22]:
def report(year=2021, delete_consecutive=False):
    display(Markdown(f"# Jahr {year} (delete_consecutive_events = {delete_consecutive})"))
    
    df = pd.read_sql(f"""
        SELECT
            event_log.id,
            event_log.actor_id,
            event_log.date,
            IF(event_log.event_id in (4,5), 1, 0) as edits,
            IF(event_log.event_id in (6,11), 1, 0) as reviews,
            IF(event_log.event_id in (8,9,14,16), 1, 0) as comments,
            IF(event_log.event_id in (1,2,12,15,17), 1, 0) as taxonomy
        FROM event_log
        WHERE event_log.event_id IN (4,5,6,11,8,9,14,16,1,2,12,15,17)
            and year(event_log.date) = {year}
    """, db)
    
    if delete_consecutive:
        df = delete_consecutive_events(df)
    
    for kind in ["edits", "reviews", "comments", "taxonomy"]:
        nr = df[kind].sum()
        display(Markdown(f"Anzahl {kind}: {nr}"))
        
    df = df.groupby("actor_id").sum()
    nr = len(df[df["edits"] > 10])
    
    display(Markdown(f"Anzahl Autor*innen mit mehr als 10 Edits: {nr}"))
    
    df["count"] = df["edits"] + df["comments"] + df["reviews"] + df["taxonomy"]
    nr = len(df[df["count"] > 10])
    
    display(Markdown(f"Anzahl Autor*innen mit mehr als 10 Events (comments, reviews, taxonomy, edits): {nr}"))
        
def delete_consecutive_events(df):
    df.sort_values(["actor_id", "date"], inplace=True)
    df["time_diff_seconds"] = df["date"].diff().map(lambda x: x.total_seconds())
    
    to_delete = (df["time_diff_seconds"] >= 0) & (df["time_diff_seconds"] < 3)
    df.drop(df[to_delete].index, inplace=True)
    
    return df

for c in [True, False]:
    for year in [2021, 2020, 2019]:
        report(year=year, delete_consecutive=c)

# Jahr 2021 (delete_consecutive_events = True)

Anzahl edits: 12674

Anzahl reviews: 9354

Anzahl comments: 941

Anzahl taxonomy: 4214

...

Anzahl reviews: 11594

Anzahl comments: 1368

Anzahl taxonomy: 3349

Anzahl Autor*innen mit mehr als 10 Edits: 95

Anzahl Autor*innen mit mehr als 10 Events (comments, reviews, taxonomy, edits): 115